In [1]:
import csv
import psycopg2

In [2]:
csv_file = "../data/Dengue_Daily_EN_aligned.csv" 
table_name = "dengue_cases"
conn_string = "postgresql://lizavabistsevits:@localhost:5432/taiwan" # database connection string

column_names = {
    "Date_Onset": "date_onset",
    "Date_Confirmation" : "date_confirmation",
    "Date_Notification": "date_notification",
    "Sex": "sex",
    "Age_Group": "age_group",
	"Imported" : "imported",
	"Country_infected" : "country_infected",
	"Number_of_confirmed_cases" : "confirmed_cases",
    "Serotype" : "serotype",
	"MOI_Township_living_Code" : "township_living",
	"MOI_Township_infected_Code" : "township_infected"
}

In [3]:
# Connect to database
with psycopg2.connect(conn_string) as conn:
    with conn.cursor() as cursor:      
        
        with open(csv_file, 'r') as file:
            csv_reader = csv.DictReader(file)
            
            # Insert data row by row
            for row in csv_reader:
                # Prepare column names and values for the specific columns
                columns = []
                values = []
                
                for csv_col, db_col in column_names.items():
                    if csv_col in row:
                        columns.append(db_col)
                        # Convert empty and None strings to NULL
                        if row[csv_col] == "" or row[csv_col] == 'None':
                            values.append(None)
                        else:
                            values.append(row[csv_col])
                
                # Create the INSERT query string with the specified columns
                columns_joined = ", ".join(columns)
                placeholders = ", ".join(["%s"] * len(values))
                insert_query = f"INSERT INTO {table_name} ({columns_joined}) VALUES ({placeholders});"
                
                # Execute the query
                cursor.execute(insert_query, values)
        
        # Commit changes
        conn.commit()

print(f"Data from {csv_file} successfully inserted to {table_name} table.")

Data from ../data/Dengue_Daily_EN_aligned.csv successfully inserted to dengue_cases table.
